In [1]:
import pathlib
import os
from collections import namedtuple
import numpy as np
import pandas as pd

# DIR_PATH = pathlib.Path(__file__).parent.resolve()
INPUT_FILE = 'test_input.txt'
INPUT_PATH = os.path.join(INPUT_FILE)

with open(INPUT_PATH, 'r') as f:
    data = f.read().split('\n')

START_IND = (0, 500)


In [ ]:
def pretty_print(input_array: np.ndarray) -> None:
    ''' Prints the screen in a pretty way'''
    A = pd.DataFrame(input_array)
    A.columns = ['']*A.shape[1]
    print(A.to_string(index=False))

In [ ]:
def min_value_position(input):
    min_val = min(input)
    return min_val, input.index(min_val)

def max_value_position(input):
    max_val = max(input)
    return max_val, input.index(max_val)

In [ ]:
def find_grain(domain):
    grain_row, grain_col = np.where(domain == "+")
    grain_row = grain_row[0]
    grain_col = grain_col[0]
    return (grain_row, grain_col)

def check_move(domain, grain_ind):
    grain_row = grain_ind[0]
    grain_col = grain_ind[1]

    below_index = (grain_row+1, grain_col)
    down_left_index = (grain_row+1, grain_col-1)
    down_right_index = (grain_row+1, grain_col+1)

    
    try:
        if domain[below_index] == ".":
            return below_index
        elif domain[down_left_index] == ".":
            return down_left_index
        elif domain[down_right_index] == ".":
            return down_right_index
        else:   
            return "next"
    except:
        return False

In [ ]:
Path = namedtuple('Path', ['x', 'y'])
paths = []
for line in data:
    path_parts = line.split(" -> ")
    xs = []
    ys = []
    for part in path_parts:
        x, y = part.split(",")
        xs.append(int(x))
        ys.append(int(y))
    paths.append(Path(x=xs, y=ys))

In [ ]:
paths_x = []
paths_y = []
for path in paths:
    for id, (x, y) in enumerate(zip(path.x[:-1], path.y[:-1])):
        x_diff = path.x[id+1] - x
        y_diff = path.y[id+1] - y
        run_length = max(abs(x_diff), abs(y_diff))
        if y_diff:
            for i in range(run_length+1):
                paths_y.append(y+(i*np.sign(y_diff)))
                paths_x.append(x)
        else:
            for i in range(run_length+1):
                paths_x.append(x+(i*np.sign(x_diff)))
                paths_y.append(y)

In [ ]:
def make_domain_part_1(paths_x, paths_y):
    x_dim = max(paths_x) - min(paths_x)
    y_dim = max(paths_y)
    domain = np.empty(shape=(y_dim+1, x_dim+1), dtype=str)
    domain[:][:] = "."
    for row, col in zip(paths_y, paths_x-min(paths_x)):
        domain[row, col] = "#"
    start_ind = (START_IND[0], START_IND[1]-min(paths_x))
    return domain, start_ind

def make_domain_part_2(paths_x, paths_y):
    domain, start_ind = make_domain_part_1(paths_x=paths_x, paths_y=paths_y)
    height = domain.shape[0] + 2
    x_min_val, x_min_index = min_value_position(paths_x)
    x_max_val, x_max_index = min_value_position(paths_x)
    x_min_val -= min(paths_x)
    x_max_val -= min(paths_x)

    distance_left = max(height,
                        x_min_val + (height - paths_y[x_min_index]))
    distance_right = max(height,
                        x_max_val + (height - paths_y[x_max_index]))
    
    add_left = abs(distance_left - start_ind[1]) + 2
    add_right = abs(distance_right - (domain.shape[1] - start_ind[1])) + 2
    print(add_left)
    left_array = np.empty(shape=(domain.shape[0], add_left), dtype=str)
    right_array = np.empty(shape=(domain.shape[0], add_right), dtype=str)
    new_width = add_left + domain.shape[1] + add_right
    bottom_array = np.empty(shape=(2, new_width), dtype=str)

    left_array[:, :] = "."
    right_array[:, :] = "."
    bottom_array[0,:] = "."
    bottom_array[1,:] = "#"
    start_ind = (start_ind[0], start_ind[1] + add_left)
    new_domain = np.append(np.append(np.append(left_array, domain, axis=1), right_array, axis=1), bottom_array, axis=0)
    return new_domain, start_ind

In [ ]:
def fill_domain(domain, start_ind):
    flag = True
    sand_ind = 1
    domain[start_ind] = "+"
    grain_ind = start_ind
    while flag:
        # try:
        #     grain_ind = find_grain(domain)
        # except:
        #     flag = False
        #     domain[start_ind] = "o"
        #     continue
        updated_index = check_move(domain, grain_ind)
        if domain[grain_ind] == "o":
            flag = False
            continue
        if updated_index == "next":
            domain[start_ind] = "+"
            domain[grain_ind] = "o"
            grain_ind = start_ind
            sand_ind += 1
            continue
        elif not updated_index:
            flag = False
            continue
        domain[updated_index] = "+"
        domain[grain_ind] = "."
        grain_ind = updated_index
    return domain, sand_ind-1


In [ ]:
#part_1
domain, start_ind = make_domain_part_1(paths_x=paths_x, paths_y=paths_y)
domain, sand_ind = fill_domain(domain, start_ind)
pretty_print(domain)
print("The solution to part 1 is ", sand_ind)

In [ ]:
#part_2
part_2_domain, start_ind = make_domain_part_2(paths_x=paths_x, paths_y=paths_y)
part_2_domain, sand_ind = fill_domain(part_2_domain, start_ind)
pretty_print(part_2_domain)
print("The solution to part 2 is", sand_ind)